# EWGCD Method: Edge-Weighted Gene Community Detection

## Library Imports

In [1]:
import pandas as pd
import networkx as nx
import itertools
import random
from networkx.algorithms.community import girvan_newman
from collections import defaultdict
from itertools import combinations
from joblib import Parallel, delayed
from multiprocessing import Pool
import matplotlib.pyplot as plt
import os

## Input: MAF File

In [2]:
maf_path = './inputs/prostate.maf'

try:
    maf_df = pd.read_csv(maf_path, sep='\t', comment='#')
    print(maf_df.head())
except Exception as e:
    print(f"Erro ao ler o arquivo MAF: {e}")

   Unnamed: 0 Hugo_Symbol  Entrez_Gene_Id         Center NCBI_Build  \
0           0     AADACL3               0  broad.mit.edu     GRCh37   
1           1    ADAMTS19               0  broad.mit.edu     GRCh37   
2           2       AGBL2               0  broad.mit.edu     GRCh37   
3           3       ASCC3               0  broad.mit.edu     GRCh37   
4           4         ATM               0  broad.mit.edu     GRCh37   

  Chromosome  Start_Position  End_Position Strand       Consequence  ...  \
0          1        12785236      12785236      +  missense_variant  ...   
1          5       129037220     129037220      +  missense_variant  ...   
2         11        47726203      47726203      +  missense_variant  ...   
3          6       101109870     101109870      +  missense_variant  ...   
4         11       108218029     108218029      +  missense_variant  ...   

  FILTER             ENSP ExAC_AF         CCDS   EXON ExAC_AF_OTH SAS_MAF  \
0      .  ENSP00000352268     NaN  CCDS

## Mutation Weights

In [3]:
mutation_weights = {
    'Nonsense_Mutation': 1.0,
    'Missense_Mutation': 0.4,
    'Splice_Site': 0.4,
    'Frame_Shift_Del': 1.0,
    'Frame_Shift_Ins': 1.0,
    'In_Frame_Del': 0.4,
    'In_Frame_Ins': 0.4,
    '3\'UTR': 0.2,
    '5\'UTR': 0.4,
    'Nonstop_Mutation': 0.4,
    'Translation_Start_Site': 0.2
}

## Output: Weighted Mutation Matrix (WMM)

In [4]:
maf_df['Weight'] = maf_df['Variant_Classification'].map(mutation_weights)

wmm = maf_df.groupby(['Tumor_Sample_Barcode', 'Hugo_Symbol'])['Weight'].mean().unstack(fill_value=0)

output_dir = './output/'
os.makedirs(output_dir, exist_ok=True)

output_file = os.path.join(output_dir, 'prostate.wmm')

wmm.to_csv(output_file, sep='\t')

print(f"Arquivo salvo em: {output_file}")

Arquivo salvo em: ./output/prostate.wmm


## Output: Weighted Frequencies

In [5]:
def calculate_normalized_gene_scores(maf_df, wmm):

    numero_pacientes = maf_df['Tumor_Sample_Barcode'].nunique()
    pontuacao_bruta_genes = wmm.sum(axis=0) / numero_pacientes
    pontuacao_normalizada_genes = pontuacao_bruta_genes / pontuacao_bruta_genes.max()

    pontuacoes_df = pd.DataFrame({
        'wf': pontuacao_bruta_genes,
        'nwf': pontuacao_normalizada_genes
    }).reset_index()

    if 'index' in pontuacoes_df.columns:
        pontuacoes_df.rename(columns={'index': 'Gene'}, inplace=True)
    elif 'Hugo_Symbol' in pontuacoes_df.columns:
        pontuacoes_df.rename(columns={'Hugo_Symbol': 'Gene'}, inplace=True)
    else:
        print("Coluna não encontrada.")

    pontuacoes_df.sort_values(by='Gene', inplace=True)
    pontuacoes_df.reset_index(drop=True, inplace=True)

    return pontuacoes_df, pontuacoes_df['nwf']

pontuacoes_df, nwf = calculate_normalized_gene_scores(maf_df, wmm)

output_file = os.path.join(output_dir, 'normalized_gene_scores.nwf')

pontuacoes_df.to_csv(output_file, sep='\t', index=False)

print(f"Arquivo salvo em: {output_file}")

Arquivo salvo em: ./output/normalized_gene_scores.nwf


## Output: Consensus Network UGN

In [6]:
def calculate_frequency_pairs(file_paths):
    exact_pair_frequency = defaultdict(int)

    for file_path in file_paths:
        with open(file_path, 'r') as file:
            for line in file:
                genes = line.strip().split()

                for i in range(len(genes) - 1):
                    pair = f"{genes[i]} {genes[i+1]}"
                    exact_pair_frequency[pair] += 1

    unordered_pair_frequency = defaultdict(int)
    for pair, freq in exact_pair_frequency.items():
        genes = pair.split()
        unordered_pair = tuple(sorted(genes))
        unordered_pair_frequency[unordered_pair] += freq

    total_files = len(file_paths)
    for pair in unordered_pair_frequency:
        unordered_pair_frequency[pair] = unordered_pair_frequency[pair] / total_files

    return unordered_pair_frequency

def create_graph(unordered_pair_frequency):
    G = nx.Graph()

    for pair, freq in unordered_pair_frequency.items():
        gene1, gene2 = pair
        G.add_edge(gene1, gene2, weight=freq)

    return G

def save_graph_to_ugn(ugn, output_file):
    with open(output_file, 'w') as file:
        for edge in ugn.edges(data=True):
            gene1, gene2, data = edge
            weight = data['weight']
            file.write(f"{gene1}\t{gene2}\t{weight}\n")
    print(f"Grafo salvo em: {output_file}")

file_paths = [
    './inputs/ReactomeFI_preprocessed_122718.txt'
]

unordered_pair_frequency = calculate_frequency_pairs(file_paths)

ugn = create_graph(unordered_pair_frequency)

output_file = './output/ugn_gene_network.ugn'
save_graph_to_ugn(ugn, output_file)

Grafo salvo em: ./output/ugn_gene_network.ugn


## Output: Gene Strength Spreading Network (GSSN)

In [8]:
def build_gssn(gene_network):
    gssn = nx.DiGraph()
    gene_network.remove_edges_from(nx.selfloop_edges(gene_network))
    
    max_weight = 0
    for g_i, g_j in gene_network.edges:
        weight_ij = gene_network[g_i][g_j]["weight"]
        r_i = sum(gene_network[g_i][neighbor]['weight'] for neighbor in gene_network[g_i])
        r_j = sum(gene_network[g_j][neighbor]['weight'] for neighbor in gene_network[g_j])
        r_i_out = sum(gene_network[g_i][neighbor]['weight'] for neighbor in set(gene_network[g_i]) - set(gene_network[g_j]) - {g_j})
        r_j_out = sum(gene_network[g_j][neighbor]['weight'] for neighbor in set(gene_network[g_j]) - set(gene_network[g_i]) - {g_i})

        s_ij = (1 + (r_i * r_j_out)) * weight_ij
        s_ji = (1 + (r_j * r_i_out)) * weight_ij

        gssn.add_edge(g_i, g_j, weight=s_ij)
        gssn.add_edge(g_j, g_i, weight=s_ji)

        max_weight = max(max_weight, s_ij, s_ji)

    for _, _, d in gssn.edges(data=True):
        d['weight'] /= max_weight

    return gssn

def save_gssn_to_file(gssn, output_file):
    with open(output_file, 'w') as file:
        for edge in gssn.edges(data=True):
            gene1, gene2, data = edge
            weight = data['weight']
            file.write(f"{gene1}\t{gene2}\t{weight:.6f}\n")
    print(f"GSSN salvo em: {output_file}")

gssn = build_gssn(ugn)

output_file = './output/gssn_gene_network.gssn'
save_gssn_to_file(gssn, output_file)

GSSN salvo em: ./output/gssn_gene_network.gssn


## Result: EWGCD Method

In [13]:
def calculate_clustering_coefficient(graph, community):
    subgraph = graph.subgraph(community)
    return nx.average_clustering(subgraph, weight='weight')

def expand_cluster(graph, seed_node, visited):
    cluster = {seed_node}
    queue = [seed_node]
    initial_coefficient = calculate_clustering_coefficient(graph, cluster)
    
    while queue:
        current = queue.pop(0)
        
        best_neighbor = None
        best_coefficient = initial_coefficient
        
        for neighbor in graph.neighbors(current):
            if neighbor not in cluster:
                potential_cluster = cluster | {neighbor}
                new_coefficient = calculate_clustering_coefficient(graph, potential_cluster)
                if new_coefficient >= best_coefficient:
                    best_coefficient = new_coefficient
                    best_neighbor = neighbor
        
        if best_neighbor is not None:
            cluster.add(best_neighbor)
            queue.append(best_neighbor)
            initial_coefficient = best_coefficient

    visited.update(cluster)
    return cluster

def detect_communities(graph):
    centrality = {node: val for node, val in graph.degree()}
    sorted_nodes = sorted(centrality, key=centrality.get, reverse=True)
    
    communities = []
    visited = set()

    for node in sorted_nodes:
        if node not in visited:
            community = expand_cluster(graph, node, visited)
            communities.append(community)

    return communities

def rank_communities(graph, communities):
    ranked_communities = []
    for community in communities:
        coefficient = calculate_clustering_coefficient(graph, community)
        ranked_communities.append((community, coefficient))
    
    ranked_communities.sort(key=lambda x: x[1], reverse=True)
    return ranked_communities

def calculate_precision(community, known_genes):
    known_count = len([gene for gene in community if gene in known_genes])
    precision = known_count / len(community) if community else 0
    return precision

def save_communities_to_file(communities, known_genes, output_file):
    data = []
    for i, (community, score) in enumerate(communities):
        precision = calculate_precision(community, known_genes)
        data.append({
            "Comunidade": ', '.join(sorted(community)),
            "Coeficiente": f"{score:.2f}",
            "Precisão (%)": f"{precision:.2}"
        })

    df = pd.DataFrame(data)
    df.sort_values(by=["Coeficiente", "Precisão (%)"], ascending=[False, False], inplace=True)
    df.to_csv(output_file, sep='\t', index=False, header=True)
    print(f"Community Result salvo em: {output_file}")

with open("./inputs/canonical_drivers.txt", 'r') as file:
    known_genes = set(line.strip() for line in file)

largest_component = max(nx.strongly_connected_components(gssn), key=len)
gssn_largest = gssn.subgraph(largest_component).copy()

initial_communities = detect_communities(gssn_largest)
ranked_communities = rank_communities(gssn_largest, initial_communities)

output_file = "./output/community_results_largest_component.output"
save_communities_to_file(ranked_communities, known_genes, output_file)

Community Result salvo em: ./output/community_results_largest_component.output


In [2]:
def filter_communities(input_file, output_file, threshold=0.20):
    df = pd.read_csv(input_file, sep='\t')
    df['Coeficiente'] = pd.to_numeric(df['Coeficiente'], errors='coerce')
    
    if df['Precisão (%)'].dtype == 'object':
        df['Precisão (%)'] = pd.to_numeric(df['Precisão (%)'].str.replace('%', ''), errors='coerce')
    else:
        df['Precisão (%)'] = df['Precisão (%)']

    filtered_df = df[(df['Coeficiente'] > threshold) & (df['Precisão (%)'] > threshold)]

    filtered_df.to_csv(output_file, sep='\t', index=False)
    print(f"Arquivo filtrado de Comunidades salvo em: {output_file}")

input_file = './output/community_results_largest_component.output'
output_file = './output/filtered_community_results.output'

filter_communities(input_file, output_file)

Arquivo filtrado de Comunidades salvo em: ./output/filtered_community_results.output
